In [1]:
import cv2
import numpy as np
import os
import pandas
import sys
import keras as K
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers import Add
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
from sklearn import preprocessing
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight


Using TensorFlow backend.


In [2]:
def read_images(filenames,directory,height,width):
    # Read Images into a Numpy array
    
    # Get the array size
    array_size = len(filenames)
    images = np.array([])
    #Preallocate the array.  This is much faster that numpy.append
    images = np.zeros((array_size,height,width))

    i=0
    for file in filenames:
        image = cv2.imread(directory + "/" + file,0)
        # Resize the image.
        resized_image = cv2.resize(image, (width,height))
        # Scale and add it to numpy array
        images[i] = resized_image/255
        i+=1

    
    
    return images

In [3]:
def get_labels(filelist,labels):
    
    labels.index = labels['Image']
    select_labels = np.array([])
    labels_filenames = []
    for file in filelist:
        select_labels = np.append(select_labels,labels.loc[file]['Encoded_Id'])
        labels_filenames.append(file)
    
    return select_labels,labels_filenames

In [4]:
def encode_labels(labels_df):

    labels = labels_df['Id'].values
    le = preprocessing.LabelEncoder()
    le.fit(labels)
    labels_encode = le.transform(labels)
    
    labels_df['Encoded_Id'] = labels_encode 
    
    return labels_df

In [ ]:
width = 128
height = 128

traindir="/home/james/Kaggle/Whale/Data/train"
labelsfile="/home/james/Kaggle/Whale/Data/train.csv"

In [ ]:
labels = pandas.read_csv(labelsfile)
labels.describe()
with pandas.option_context('display.max_rows', None, 'display.max_columns', None):
    print(labels['Id'].value_counts())

new_whale    9664
w_23a388d      73
w_9b5109b      65
w_9c506f6      62
w_0369a5c      61
w_700ebb4      57
w_3de579a      54
w_564a34b      51
w_fd3e556      50
w_88e4537      49
w_2b069ba      48
w_d405854      47
w_789c969      45
w_f0fe284      45
w_778e474      40
w_343f088      40
w_5e8e218      40
w_60ce6fc      37
w_a9304b9      37
w_5a2634c      37
w_6822dbc      36
w_af367c3      35
w_1ca9ab1      34
w_f765256      34
w_17b0d3a      33
w_d72771c      32
w_6cda039      31
w_08630fd      31
w_8c25681      31
w_51fc1fc      30
w_04003e9      30
w_fd6a3ad      30
w_2365d55      29
w_59052ad      29
w_6caff0f      27
w_91cc02c      25
w_bfcad53      25
w_75f6ffa      24
w_27597ff      24
w_b9e5911      24
w_0bc078c      24
w_0a155b9      24
w_990921b      24
w_cd4cb49      24
w_0135f5f      23
w_5773c71      23
w_e1ffbe2      23
w_242fb46      23
w_8da30ad      22
w_f61b957      22
w_60cf87c      22
w_488865e      22
w_efbdcbc      22
w_e2a09d4      21
w_5597026      21
w_bc285a6 

In [ ]:

labels = pandas.read_csv(labelsfile)

y=labels['Id']
values = np.array(y)

class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(values),
                                                 values)

print(class_weights)

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
train_labels = onehot_encoded


[5.24330802e-04 5.06713287e+00 5.06713287e+00 ... 5.06713287e+00
 5.06713287e+00 5.06713287e+00]


In [ ]:
#Get List of Directories
directories1 = os.listdir(traindir)  

directories = []
for i in range(20):
    directories = directories + directories1

    
dropout = 0.25
    
# Get number of classes
num_classes = train_labels.shape[1]
input_shape=(height,width,1)
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(dropout))

model.add(Conv2D(32, kernel_size=(3, 3),activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(dropout))

model.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(dropout))

model.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(dropout))

model.add(Conv2D(128, (3, 3), activation='relu',padding='same'))
model.add(Dropout(dropout))
model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout))

model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=["accuracy"],
              #metrics=[mean_pred]
             )


#Save Models
filepath="/home/james/Kaggle/Whale/Data/whales-copy" + ".best.hdf5"

checkpoint = ModelCheckpoint(filepath, 
                             monitor="acc",
                             verbose=1, 
                             save_best_only=True, 
                             mode='max')


callbacks_list = [checkpoint]


In [ ]:
directory="masked_img"

filenames = os.listdir(traindir+"/" + directory)
filenames.sort()

chunk_images = read_images(filenames,traindir + "/" + directory,height,width)

# Reshape Array
chunk_images = chunk_images.reshape((chunk_images.shape[0],height,width,1))

In [ ]:
#model.load_weights(filepath)


datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    featurewise_center=True, 
    featurewise_std_normalization=True
    )

datagen.fit(chunk_images)

history = model.fit_generator(
                    datagen.flow(chunk_images, train_labels, batch_size=10),
                    steps_per_epoch=len(chunk_images)/10, 
                    epochs=1500,
                    callbacks=callbacks_list,
                    )

#history = model.fit(chunk_images,train_labels,
#                    #X_train, y_train,
#                    epochs=1500,
#                    callbacks=callbacks_list,
#                    class_weight=class_weights,
#                    #validation_data=(X_test,y_test),
#                    batch_size=100,
#                    verbose=1,
#                    shuffle=True)

Epoch 1/1500
2537/2536 [==============================] - 36s 14ms/step - loss: 6.1478 - acc: 0.3689

Epoch 00001: acc improved from -inf to 0.36899, saving model to /home/james/Kaggle/Whale/Data/whales-copy.best.hdf5
Epoch 2/1500
2537/2536 [==============================] - 32s 13ms/step - loss: 5.6608 - acc: 0.3808

Epoch 00002: acc improved from 0.36899 to 0.38094, saving model to /home/james/Kaggle/Whale/Data/whales-copy.best.hdf5
Epoch 3/1500
2537/2536 [==============================] - 31s 12ms/step - loss: 5.5603 - acc: 0.3808

Epoch 00003: acc improved from 0.38094 to 0.38098, saving model to /home/james/Kaggle/Whale/Data/whales-copy.best.hdf5
Epoch 4/1500
2537/2536 [==============================] - 31s 12ms/step - loss: 5.4741 - acc: 0.3809

Epoch 00004: acc improved from 0.38098 to 0.38102, saving model to /home/james/Kaggle/Whale/Data/whales-copy.best.hdf5
Epoch 5/1500
2537/2536 [==============================] - 32s 13ms/step - loss: 5.3953 - acc: 0.3807

Epoch 00005: acc 

Epoch 53/1500
2537/2536 [==============================] - 42s 16ms/step - loss: 4.0122 - acc: 0.3812

Epoch 00053: acc did not improve from 0.38181
Epoch 54/1500
2537/2536 [==============================] - 34s 13ms/step - loss: 4.0052 - acc: 0.3821

Epoch 00054: acc improved from 0.38181 to 0.38220, saving model to /home/james/Kaggle/Whale/Data/whales-copy.best.hdf5
Epoch 55/1500
2537/2536 [==============================] - 35s 14ms/step - loss: 3.9629 - acc: 0.3853

Epoch 00055: acc improved from 0.38220 to 0.38504, saving model to /home/james/Kaggle/Whale/Data/whales-copy.best.hdf5
Epoch 56/1500
2537/2536 [==============================] - 43s 17ms/step - loss: 3.9763 - acc: 0.3816

Epoch 00056: acc did not improve from 0.38504
Epoch 57/1500
2537/2536 [==============================] - 31s 12ms/step - loss: 3.9708 - acc: 0.3816

Epoch 00057: acc did not improve from 0.38504
Epoch 58/1500
2537/2536 [==============================] - 37s 15ms/step - loss: 3.9576 - acc: 0.3850

Epoch 

Epoch 104/1500
2537/2536 [==============================] - 45s 18ms/step - loss: 3.6998 - acc: 0.3934

Epoch 00104: acc improved from 0.39297 to 0.39352, saving model to /home/james/Kaggle/Whale/Data/whales-copy.best.hdf5
Epoch 105/1500
2537/2536 [==============================] - 31s 12ms/step - loss: 3.6683 - acc: 0.3948

Epoch 00105: acc improved from 0.39352 to 0.39458, saving model to /home/james/Kaggle/Whale/Data/whales-copy.best.hdf5
Epoch 106/1500
2537/2536 [==============================] - 43s 17ms/step - loss: 3.6873 - acc: 0.3907

Epoch 00106: acc did not improve from 0.39458
Epoch 107/1500
2537/2536 [==============================] - 38s 15ms/step - loss: 3.6786 - acc: 0.3934

Epoch 00107: acc did not improve from 0.39458
Epoch 108/1500
2537/2536 [==============================] - 39s 15ms/step - loss: 3.8423 - acc: 0.3896

Epoch 00108: acc did not improve from 0.39458
Epoch 109/1500
2537/2536 [==============================] - 43s 17ms/step - loss: 3.9342 - acc: 0.3849



Epoch 156/1500
2537/2536 [==============================] - 40s 16ms/step - loss: 3.5525 - acc: 0.3996

Epoch 00156: acc improved from 0.39935 to 0.39935, saving model to /home/james/Kaggle/Whale/Data/whales-copy.best.hdf5
Epoch 157/1500
2537/2536 [==============================] - 34s 13ms/step - loss: 3.5520 - acc: 0.3976

Epoch 00157: acc did not improve from 0.39935
Epoch 158/1500
2537/2536 [==============================] - 45s 18ms/step - loss: 3.5446 - acc: 0.3979

Epoch 00158: acc did not improve from 0.39935
Epoch 159/1500
2537/2536 [==============================] - 31s 12ms/step - loss: 3.5461 - acc: 0.3979

Epoch 00159: acc did not improve from 0.39935
Epoch 160/1500
2537/2536 [==============================] - 39s 15ms/step - loss: 3.5444 - acc: 0.3979

Epoch 00160: acc did not improve from 0.39935
Epoch 161/1500
2537/2536 [==============================] - 40s 16ms/step - loss: 3.5492 - acc: 0.3967

Epoch 00161: acc did not improve from 0.39935
Epoch 162/1500
2537/2536 [=

2537/2536 [==============================] - 35s 14ms/step - loss: 3.4668 - acc: 0.4025

Epoch 00207: acc did not improve from 0.40740
Epoch 208/1500
2537/2536 [==============================] - 31s 12ms/step - loss: 3.4649 - acc: 0.4020

Epoch 00208: acc did not improve from 0.40740
Epoch 209/1500
2537/2536 [==============================] - 33s 13ms/step - loss: 3.4648 - acc: 0.4015

Epoch 00209: acc did not improve from 0.40740
Epoch 210/1500
2537/2536 [==============================] - 35s 14ms/step - loss: 3.4575 - acc: 0.4043

Epoch 00210: acc did not improve from 0.40740
Epoch 211/1500
2537/2536 [==============================] - 31s 12ms/step - loss: 3.4713 - acc: 0.4038

Epoch 00211: acc did not improve from 0.40740
Epoch 212/1500
2537/2536 [==============================] - 32s 13ms/step - loss: 3.4601 - acc: 0.4034

Epoch 00212: acc did not improve from 0.40740
Epoch 213/1500
2537/2536 [==============================] - 36s 14ms/step - loss: 3.4452 - acc: 0.4058

Epoch 00213

...

2537/2536 [==============================] - 31s 12ms/step - loss: 3.2116 - acc: 0.4202

Epoch 00422: acc did not improve from 0.42254
Epoch 423/1500
2537/2536 [==============================] - 37s 15ms/step - loss: 3.1843 - acc: 0.4206

Epoch 00423: acc did not improve from 0.42254
Epoch 424/1500
2537/2536 [==============================] - 31s 12ms/step - loss: 3.1802 - acc: 0.4240

Epoch 00424: acc improved from 0.42254 to 0.42419, saving model to /home/james/Kaggle/Whale/Data/whales-copy.best.hdf5
Epoch 425/1500
2537/2536 [==============================] - 33s 13ms/step - loss: 3.1717 - acc: 0.4231

Epoch 00425: acc did not improve from 0.42419
Epoch 426/1500
2537/2536 [==============================] - 36s 14ms/step - loss: 3.1970 - acc: 0.4201

Epoch 00426: acc did not improve from 0.42419
Epoch 427/1500
2537/2536 [==============================] - 31s 12ms/step - loss: 3.1824 - acc: 0.4210

Epoch 00427: acc did not improve from 0.42419
Epoch 428/1500
2537/2536 [================

2537/2536 [==============================] - 50s 20ms/step - loss: 3.1322 - acc: 0.4259

Epoch 00475: acc did not improve from 0.42727
Epoch 476/1500
2537/2536 [==============================] - 31s 12ms/step - loss: 3.1316 - acc: 0.4250

Epoch 00476: acc did not improve from 0.42727
Epoch 477/1500
2537/2536 [==============================] - 42s 17ms/step - loss: 3.1375 - acc: 0.4213

Epoch 00477: acc did not improve from 0.42727
Epoch 478/1500
2537/2536 [==============================] - 42s 17ms/step - loss: 3.1223 - acc: 0.4305

Epoch 00478: acc improved from 0.42727 to 0.43070, saving model to /home/james/Kaggle/Whale/Data/whales-copy.best.hdf5
Epoch 479/1500
2537/2536 [==============================] - 32s 13ms/step - loss: 3.1379 - acc: 0.4237

Epoch 00479: acc did not improve from 0.43070
Epoch 480/1500
2537/2536 [==============================] - 44s 17ms/step - loss: 3.1405 - acc: 0.4261

Epoch 00480: acc did not improve from 0.43070
Epoch 481/1500
2537/2536 [================

2537/2536 [==============================] - 44s 17ms/step - loss: 3.1148 - acc: 0.4249

Epoch 00529: acc did not improve from 0.43070
Epoch 530/1500
2537/2536 [==============================] - 31s 12ms/step - loss: 3.1045 - acc: 0.4271

Epoch 00530: acc did not improve from 0.43070
Epoch 531/1500
2537/2536 [==============================] - 40s 16ms/step - loss: 3.1159 - acc: 0.4233

Epoch 00531: acc did not improve from 0.43070
Epoch 532/1500
2537/2536 [==============================] - 39s 15ms/step - loss: 3.1175 - acc: 0.4256

Epoch 00532: acc did not improve from 0.43070
Epoch 533/1500
2537/2536 [==============================] - 34s 13ms/step - loss: 3.1160 - acc: 0.4224

Epoch 00533: acc did not improve from 0.43070
Epoch 534/1500
2537/2536 [==============================] - 46s 18ms/step - loss: 3.1137 - acc: 0.4266

Epoch 00534: acc did not improve from 0.43070
Epoch 535/1500
2537/2536 [==============================] - 34s 13ms/step - loss: 3.1085 - acc: 0.4259

Epoch 00535

2537/2536 [==============================] - 35s 14ms/step - loss: 3.0768 - acc: 0.4267

Epoch 00583: acc did not improve from 0.43287
Epoch 584/1500
2537/2536 [==============================] - 46s 18ms/step - loss: 3.0745 - acc: 0.4304

Epoch 00584: acc did not improve from 0.43287
Epoch 585/1500
2537/2536 [==============================] - 31s 12ms/step - loss: 3.0729 - acc: 0.4297

Epoch 00585: acc did not improve from 0.43287
Epoch 586/1500
2537/2536 [==============================] - 41s 16ms/step - loss: 3.0730 - acc: 0.4271

Epoch 00586: acc did not improve from 0.43287
Epoch 587/1500
2537/2536 [==============================] - 43s 17ms/step - loss: 3.0619 - acc: 0.4319

Epoch 00587: acc did not improve from 0.43287
Epoch 588/1500
2537/2536 [==============================] - 35s 14ms/step - loss: 3.0602 - acc: 0.4314

Epoch 00588: acc did not improve from 0.43287
Epoch 589/1500
2537/2536 [==============================] - 48s 19ms/step - loss: 3.0630 - acc: 0.4292

Epoch 00589

2537/2536 [==============================] - 40s 16ms/step - loss: 3.0271 - acc: 0.4318

Epoch 00637: acc did not improve from 0.43437
Epoch 638/1500
2537/2536 [==============================] - 43s 17ms/step - loss: 3.0485 - acc: 0.4277

Epoch 00638: acc did not improve from 0.43437
Epoch 639/1500
2537/2536 [==============================] - 35s 14ms/step - loss: 3.0353 - acc: 0.4326

Epoch 00639: acc did not improve from 0.43437
Epoch 640/1500
2537/2536 [==============================] - 45s 18ms/step - loss: 3.0512 - acc: 0.4290

Epoch 00640: acc did not improve from 0.43437
Epoch 641/1500
2537/2536 [==============================] - 32s 13ms/step - loss: 3.0492 - acc: 0.4312

Epoch 00641: acc did not improve from 0.43437
Epoch 642/1500
2537/2536 [==============================] - 54s 21ms/step - loss: 3.0446 - acc: 0.4299

Epoch 00642: acc did not improve from 0.43437
Epoch 643/1500
 245/2536 [=>............................] - ETA: 28s - loss: 2.7790 - acc: 0.4473

In [12]:
testdir = "/home/james/Kaggle/Whale/Data/test/masked_img"
testfilenames = os.listdir(testdir)
testfilenames.sort()


test_images = read_images(testfilenames,testdir,height,width)
# Reshape Array
test_images = test_images.reshape((test_images.shape[0],height,width,1))


In [13]:


model.load_weights(filepath)
# Make prediction
Predictions = model.predict(test_images)

In [14]:
test_csv = "/home/james/Kaggle/Whale/Data/test.csv"

test = testfilenames
col = ['Image']
test_df = pandas.DataFrame(test, columns=col)
test_df['Id'] = ''

for i, pred in enumerate(Predictions):
    ids = label_encoder.inverse_transform(pred.argsort()[-5:][::-1])
    ids = ids.tolist()
    sorted_idx = pred.argsort()[-5:][::-1]
    sorted_values = []
    for idx in sorted_idx:
        sorted_values.append(pred[idx])

    print(sorted_values)
    #if sorted_values[0] < .40:
    #    ids.pop(-1)
    #    ids.insert(0,'new_whale')
    
    test_df.loc[i, 'Id'] = ' '.join(ids)

print(test_df.head(10))

test_df.to_csv(test_csv, index=False)

/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Re

[0.8249464, 0.073061496, 0.02139748, 0.021282008, 0.010193502]
[0.6127838, 0.22906947, 0.10373052, 0.021471515, 0.013009403]
[0.82276, 0.028861862, 0.016872264, 0.011450976, 0.0069012097]
[0.7616223, 0.06900632, 0.060773242, 0.04051692, 0.037301026]
[0.9942179, 0.0020171925, 0.0012025844, 0.00022937915, 0.00015833329]
[0.8621996, 0.07122691, 0.014680684, 0.013586378, 0.0034056208]
[0.45560434, 0.35086378, 0.1513103, 0.008576351, 0.0060373307]
[0.9480491, 0.01633631, 0.011351543, 0.0026266158, 0.0015946158]
[0.9615303, 0.013479052, 0.005234389, 0.0051918426, 0.0013475858]
[0.9969592, 0.00040271244, 0.00036137176, 0.00029202356, 0.00019119693]
[0.9888392, 0.001600719, 0.0015452543, 0.001529241, 0.0011741546]
[0.8901869, 0.0720919, 0.034187283, 0.0012698954, 0.0005125515]
[0.9919715, 0.002722509, 0.0017635104, 0.00096765754, 0.0002875071]
[0.9235773, 0.01291335, 0.012641816, 0.012108223, 0.0083617885]
[0.6090143, 0.22329897, 0.102540314, 0.011176294, 0.010677378]
[0.9969246, 0.001652453, 

/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Re

[0.8616522, 0.07294285, 0.025602689, 0.0075972574, 0.006243076]
[0.9785891, 0.007379311, 0.0036312975, 0.0021139006, 0.0013258825]
[0.9187278, 0.018361198, 0.006956057, 0.0039212285, 0.0036519347]
[0.97799677, 0.0046652514, 0.0027777136, 0.0014688213, 0.0012119133]
[0.6954373, 0.26102686, 0.019153556, 0.004953428, 0.003787093]
[0.99982363, 6.505014e-05, 4.4210956e-05, 2.0044965e-05, 5.2676164e-06]
[0.99766016, 0.00043023372, 0.0001897691, 0.00016561002, 0.00016255838]
[0.20441255, 0.06140271, 0.044376954, 0.04137791, 0.039602395]
[0.60977495, 0.26974142, 0.04518919, 0.014346633, 0.007955019]
[0.6140822, 0.32537115, 0.015637962, 0.012263637, 0.004590162]
[0.42753008, 0.19555953, 0.13900933, 0.08453096, 0.04749795]
[0.9396394, 0.027716402, 0.024484703, 0.004644367, 0.0022687588]
[0.93771124, 0.013713079, 0.00877684, 0.007194557, 0.00450461]
[0.89233345, 0.0393107, 0.014152487, 0.010131491, 0.0091335075]
[0.78836924, 0.052149557, 0.034337707, 0.018078834, 0.015860004]
[0.44287726, 0.36497

/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Re

...

[0.99812704, 0.00043511784, 8.544298e-05, 6.796042e-05, 6.386873e-05]
[0.9889654, 0.0014690394, 0.000901015, 0.0006881966, 0.00044759692]
[0.8163572, 0.06804141, 0.014850836, 0.010877482, 0.010146995]
[0.9927523, 0.003730035, 0.00034960077, 0.00033589048, 0.00031109378]
[0.4015294, 0.24709591, 0.15912807, 0.06706442, 0.018825326]
[0.8217463, 0.10178492, 0.020616127, 0.011690545, 0.010245647]
[0.7660246, 0.15815863, 0.02922467, 0.009063678, 0.007587604]
[0.8610308, 0.04301801, 0.019501075, 0.017869236, 0.009413751]
[0.75374794, 0.13448554, 0.041308932, 0.011004295, 0.009704776]
[0.98401284, 0.0045784675, 0.0030408266, 0.0016308954, 0.0008920519]
[0.8635146, 0.11314474, 0.0063957293, 0.0030120092, 0.0013152497]
[0.90196514, 0.03398392, 0.012262679, 0.008364973, 0.0068258974]
[0.97617763, 0.0062101777, 0.0025858805, 0.0025727255, 0.0010505932]
[0.43457928, 0.41897714, 0.069651015, 0.04717878, 0.005996484]
[0.9997483, 0.0002028534, 1.4747016e-05, 1.0938879e-05, 3.7052487e-06]
[0.31927362, 

/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Re

[0.2922568, 0.27756372, 0.08287624, 0.048209704, 0.046921674]
[0.739834, 0.11972367, 0.11118978, 0.0066934302, 0.004726577]
[0.7866511, 0.092097275, 0.049018685, 0.017217483, 0.012357018]
[0.48128578, 0.43875176, 0.013419505, 0.00917002, 0.0077764853]
[0.47311953, 0.2821991, 0.10108961, 0.02441465, 0.022494745]
[0.9679455, 0.0023735843, 0.001762405, 0.0013688551, 0.0013022453]
[0.9910563, 0.0037745151, 0.00045505617, 0.00043023992, 0.0004228714]
[0.9860908, 0.0055200136, 0.0018153397, 0.00097614544, 0.0007354008]
[0.2627469, 0.23901574, 0.11642478, 0.10644767, 0.05437167]
[0.44586536, 0.0717063, 0.06833838, 0.06749044, 0.040059593]
[0.53095835, 0.17072324, 0.048002657, 0.02874034, 0.012836227]
[0.42770997, 0.24187909, 0.22651386, 0.04338639, 0.012857371]
[0.6624684, 0.2080728, 0.023426121, 0.017473817, 0.015240528]
[0.93911207, 0.012089675, 0.011030537, 0.009167425, 0.004624237]
[0.36672106, 0.152994, 0.075490914, 0.062344212, 0.040572483]
[0.95819825, 0.005995605, 0.00451341, 0.004398

/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/Anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Re

[0.40529007, 0.26711607, 0.095488735, 0.035292476, 0.033268742]
[0.42635995, 0.3178407, 0.08744346, 0.06298737, 0.056209758]
[0.73623544, 0.06752221, 0.031818796, 0.022449747, 0.018630212]
[0.9571497, 0.02062459, 0.008499979, 0.0024585738, 0.0013091962]
[0.7132116, 0.11403084, 0.048191693, 0.036826864, 0.0072124093]
[0.74052817, 0.13581659, 0.10154707, 0.008102622, 0.0022800767]
[0.9737114, 0.014550656, 0.0026903586, 0.002452402, 0.0016402649]
[0.99507385, 0.0013127152, 0.0009951682, 0.0004220713, 0.0001899663]
[0.9891643, 0.009537799, 0.0003397991, 0.00016303148, 7.4638265e-05]
[0.854529, 0.103085846, 0.0075585363, 0.0033406601, 0.0028477546]
[0.9732483, 0.0071869325, 0.0023378257, 0.001533166, 0.0013718855]
[0.9988655, 0.00072514586, 0.00014972681, 0.000102526144, 2.8954038e-05]
[0.51970685, 0.21235365, 0.04541477, 0.034902234, 0.034312397]
[0.94197714, 0.022057638, 0.017557178, 0.015929198, 0.0009191015]
[0.965715, 0.02212026, 0.0065658432, 0.001108705, 0.0007849354]
[0.9954705, 0.0